In [2]:
import pandas as pd
import numpy as np
import re

# Major Crime Data
Source: https://data.torontopolice.on.ca/datasets/TorontoPS::major-crime-indicators-open-data/about

In [20]:
crime = pd.read_csv('data/major_crime.csv', index_col='OBJECTID')
crime.head()

,X,Y,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,REPORT_DOW,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,-8.809036e+06,5.431523e+06,GO-20141260264,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,Wednesday,...,1430,100,Assault,Assault,143,West Rouge,131,Rouge (131),-79.132915,43.780413
2,-8.814320e+06,5.435514e+06,GO-20141260033,2014/01/01 05:00:00+00,2013/12/31 05:00:00+00,2014,January,1,1,Wednesday,...,1430,100,Assault,Assault,144,Morningside Heights,131,Rouge (131),-79.180387,43.806289
3,-8.832825e+06,5.419631e+06,GO-20141259834,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,Wednesday,...,1420,100,Assault With Weapon,Assault,55,Thorncliffe Park,55,Thorncliffe Park (55),-79.346615,43.703234
4,-8.847292e+06,5.429042e+06,GO-20141264084,2014/01/01 05:00:00+00,2013/12/31 05:00:00+00,2014,January,1,1,Wednesday,...,1420,110,Assault Bodily Harm,Assault,27,York University Heights,27,York University Heights (27),-79.476579,43.764317
5,6.327780e-09,5.664924e-09,GO-20141260577,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,Wednesday,...,1610,180,Robbery - Swarming,Robbery,NSA,NSA,NSA,NSA,0.000000,0.000000


In [21]:
crime.columns

Index(['X', 'Y', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE', 'REPORT_YEAR',
       'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR',
       'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR',
       'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT',
       'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [22]:
# The OFFENCE column has very specific descriptions of the crimes, we are only interested in the larger category
# As for the report date columns, we are interested in when it happened - not when it was reported.
crime.drop(columns=['REPORT_DATE', 'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR', 'OFFENCE', 'HOOD_158', 'HOOD_140'], inplace=True)

In [23]:
# Create a new DataFrame with the unique neighbourhood158 values and their corresponding updated names
neighbourhood_mapping = crime.groupby('NEIGHBOURHOOD_158')['NEIGHBOURHOOD_140'].unique().reset_index()

new_neighbourhood_df = pd.DataFrame({
    'past_neighbourhood_158': neighbourhood_mapping['NEIGHBOURHOOD_158'],
    'neighbourhood': neighbourhood_mapping['NEIGHBOURHOOD_140'].apply(lambda x: x[0]) # assuming there's only one updated name
})

new_neighbourhood_df

,past_neighbourhood_158,neighbourhood
0,Agincourt North,Agincourt North (129)
1,Agincourt South-Malvern West,Agincourt South-Malvern West (128)
2,Alderwood,Alderwood (20)
3,Annex,Annex (95)
4,Avondale,Willowdale East (51)
...,...,...
154,Yonge-Doris,Willowdale East (51)
155,Yonge-Eglinton,Yonge-Eglinton (100)
156,Yonge-St.Clair,Yonge-St.Clair (97)
157,York University Heights,York University Heights (27)


In [24]:
# Remove the number and brackets at the end of the neighbourhood name
def clean_string(text):
    pattern = r'\s*\(\d+\)$'
    cleaned = re.sub(pattern, '', text)

    return cleaned

In [25]:
new_neighbourhood_df['neighbourhood'] = new_neighbourhood_df['neighbourhood'].apply(clean_string)

In [26]:
crime['NEIGHBOURHOOD_140'] = crime['NEIGHBOURHOOD_140'].apply(clean_string)

In [27]:
# Now we can drop the old neighbourhood_158 column
crime.drop(columns=['NEIGHBOURHOOD_158'], inplace=True)

In [28]:
crime.columns

Index(['X', 'Y', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR', 'OCC_MONTH',
       'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR', 'DIVISION',
       'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT', 'MCI_CATEGORY',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [29]:
# Converting the occurence data to a datetime variable for easy filtering
crime['OCC_DATE'] = pd.to_datetime(crime['OCC_DATE'], format='%Y/%m/%d %H:%M:%S+00')

In [30]:
# Filtering out crimes that happened before 01/01/2013 and after 31/12/2023
start_date = pd.to_datetime('2013-01-01')
end_date = pd.to_datetime('2023-12-31')

crime = crime[(crime['OCC_DATE'] >= start_date) & (crime['OCC_DATE'] <= end_date)]

In [31]:
crime = crime.sort_values(by='OCC_DATE')

In [33]:
# Create separate data frames for each year
crime_by_date = crime.groupby(crime['OCC_DATE'].dt.year)
yearly_crime_dfs = {year: group for year, group in crime_by_date}
# access each year by -> yearly_crime_dfs[x] where x is 2013, 2014, ..., 2023

In [34]:
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [18]:
monthly_summaries = {}
crime_types = ['Assault', 'Break and Enter', 'Theft Over', 'Robbery', 'Auto Theft']

# Iterate through each yearly DataFrame
for year, df_yearly in yearly_crime_dfs.items():
    df_yearly['MONTH'] = df_yearly['OCC_DATE'].dt.month
    
    # Group the yearly DataFrame by month and by crime category, then count the number of crimes
    monthly_summary_yearly = df_yearly.groupby(['MONTH', 'MCI_CATEGORY']).size().unstack(fill_value=0).reset_index()
    
    # Add a 'YEAR' column to the monthly summary DataFrame
    monthly_summary_yearly['YEAR'] = year
    monthly_summary_yearly = monthly_summary_yearly[['YEAR', 'MONTH'] + crime_types]
    
    monthly_summaries[year] = monthly_summary_yearly.to_dict(orient='records')

In [106]:
# crimes by neighbourhood
yearly_neighborhood_summaries = {}

for year, df_yearly in yearly_crime_dfs.items():
    neighborhood_summary = df_yearly['NEIGHBOURHOOD_140'].value_counts().reset_index()
    neighborhood_summary.columns = ['NEIGHBOURHOOD_140', 'num_crimes']
    
    yearly_neighborhood_summaries[year] = neighborhood_summary

In [108]:
# save as .csv
for year in years:
    yearly_crime_dfs[year].to_csv('data/crime/by_year/all_crimes' + str(year) + '.csv')
    pd.DataFrame(monthly_summaries[year]).to_csv('data/crime/by_year_month/crime_summary' + str(year) + '.csv')
    yearly_neighborhood_summaries[year].to_csv('data/crime/by_neighbourhood/' + str(year) + 'crimes_per_neighbourhood.csv')

In [35]:
crime.to_csv('data/crime/major_crimes.csv')
new_neighbourhood_df.to_csv('data/neighbourhoods.csv')

# Neighbourhood Profile Data
Source: https://open.toronto.ca/dataset/neighbourhood-profiles/

In [41]:
neighbourhoods = pd.read_csv('data/neighbourhoods_140.csv', index_col=0)
neighbourhoods.head(2)

,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,CLASSIFICATION,CLASSIFICATION_CODE,OBJECTID,geometry
_id,,,,,,,,,,,
1,2501286,26022481,NaN,30,30,Brookhaven-Amesbury (30),Brookhaven-Amesbury (30),Not an NIA or Emerging Neighbourhood,NaN,17818369,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,2501285,26022480,NaN,106,106,Humewood-Cedarvale (106),Humewood-Cedarvale (106),Not an NIA or Emerging Neighbourhood,NaN,17818385,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."


In [42]:
neighbourhoods.columns

Index(['AREA_ID', 'AREA_ATTR_ID', 'PARENT_AREA_ID', 'AREA_SHORT_CODE',
       'AREA_LONG_CODE', 'AREA_NAME', 'AREA_DESC', 'CLASSIFICATION',
       'CLASSIFICATION_CODE', 'OBJECTID', 'geometry'],
      dtype='object')

In [43]:
neighbourhoods.drop(columns=['PARENT_AREA_ID', 'AREA_SHORT_CODE', 'AREA_LONG_CODE', 'AREA_DESC', 'CLASSIFICATION_CODE'], inplace=True)

In [45]:
neighbourhoods['AREA_NAME'] = neighbourhoods['AREA_NAME'].apply(clean_string)

In [47]:
neighbourhoods.to_csv('data/neighbourhoods_140.csv')

# Neighbourhood Profile 2016

In [123]:
census2016 = pd.read_csv('data/neighbourhood-profiles-2016.csv', index_col=0)

In [124]:
census2016.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
_id,,,,,,,,,,,,,,,,,,,,,
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,...,37,7,137,64,60,94,100,97,27,31
2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,2.90%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [125]:
# drop the following columns as they do not exist in the census 2021 file
census2016.drop(columns=['Category', 'Topic', 'Data Source'], inplace=True)

In [126]:
# Now transpose the dataframe so the columns are rows and vice versa
census2016.set_index('Characteristic', inplace=True)
census2016 = census2016.transpose()
census2016.head()

Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Children (0-14 years),...,Intraprovincial migrants,Interprovincial migrants,External migrants,Total - Mobility status 5 years ago - 25% sample data,Non-movers,Movers,Non-migrants,Migrants,Interprovincial migrants,External migrants
City of Toronto,NaN,NaN,"2,731,571","2,615,060",4.50%,"1,179,057","1,112,929","4,334",630.2,"398,135",...,"49,990","15,780","59,945","2,556,120","1,516,110","1,040,015","639,060","400,950","42,985","216,835"
Agincourt North,129,No Designation,"29,113","30,279",-3.90%,"9,371","9,120","3,929",7.41,"3,840",...,275,75,605,"27,490","18,865","8,610","5,445","3,170",135,"2,280"
Agincourt South-Malvern West,128,No Designation,"23,757","21,988",8.00%,"8,535","8,136","3,034",7.83,"3,075",...,320,90,490,"22,325","13,565","8,775","5,610","3,145",220,"2,170"
Alderwood,20,No Designation,"12,054","11,904",1.30%,"4,732","4,616","2,435",4.95,"1,760",...,220,40,70,"11,370","8,235","3,130","2,200",925,70,245
Annex,95,No Designation,"30,526","29,177",4.60%,"18,109","15,934","10,863",2.81,"2,360",...,900,385,835,"27,715","12,980","14,735","8,340","6,390","1,310","2,460"


In [127]:
for col in census2016.columns:
    print(col)

Neighbourhood Number
TSNS2020 Designation
Population, 2016
Population, 2011
Population Change 2011-2016
Total private dwellings
Private dwellings occupied by usual residents
Population density per square kilometre
Land area in square kilometres
Children (0-14 years)
Youth (15-24 years)
Working Age (25-54 years)
Pre-retirement (55-64 years)
Seniors (65+ years)
Older Seniors (85+ years)
Male: 0 to 04 years
Male: 05 to 09 years
Male: 10 to 14 years
Male: 15 to 19 years
Male: 20 to 24 years
Male: 25 to 29 years
Male: 30 to 34 years
Male: 35 to 39 years
Male: 40 to 44 years
Male: 45 to 49 years
Male: 50 to 54 years
Male: 55 to 59 years
Male: 60 to 64 years
Male: 65 to 69 years
Male: 70 to 74 years
Male: 75 to 79 years
Female: 10 to 14 years
Male: 80 to 84 years
Male: 85 to 89 years
Male: 90 to 94 years
Male: 95 to 99 years
Male: 100 years and over
Female: 0 to 04 years
Female: 05 to 09 years
Female: 15 to 19 years
Female: 20 to 24 years
Female: 25 to 29 years
Female: 30 to 34 years
Female: 

In [128]:
# There are lots of trailing white spaces which will affect the selection of columns so that is being taken out
census2016.columns = [col.strip() for col in census2016.columns]

In [129]:
for col in census2016.columns:
    print(col)

Neighbourhood Number
TSNS2020 Designation
Population, 2016
Population, 2011
Population Change 2011-2016
Total private dwellings
Private dwellings occupied by usual residents
Population density per square kilometre
Land area in square kilometres
Children (0-14 years)
Youth (15-24 years)
Working Age (25-54 years)
Pre-retirement (55-64 years)
Seniors (65+ years)
Older Seniors (85+ years)
Male: 0 to 04 years
Male: 05 to 09 years
Male: 10 to 14 years
Male: 15 to 19 years
Male: 20 to 24 years
Male: 25 to 29 years
Male: 30 to 34 years
Male: 35 to 39 years
Male: 40 to 44 years
Male: 45 to 49 years
Male: 50 to 54 years
Male: 55 to 59 years
Male: 60 to 64 years
Male: 65 to 69 years
Male: 70 to 74 years
Male: 75 to 79 years
Female: 10 to 14 years
Male: 80 to 84 years
Male: 85 to 89 years
Male: 90 to 94 years
Male: 95 to 99 years
Male: 100 years and over
Female: 0 to 04 years
Female: 05 to 09 years
Female: 15 to 19 years
Female: 20 to 24 years
Female: 25 to 29 years
Female: 30 to 34 years
Female: 

In [130]:
pop_by_age_group = [
    'Male: 0 to 04 years',
    'Male: 05 to 09 years',
    'Male: 10 to 14 years',
    'Male: 15 to 19 years',
    'Male: 20 to 24 years',
    'Male: 25 to 29 years',
    'Male: 30 to 34 years',
    'Male: 35 to 39 years',
    'Male: 40 to 44 years',
    'Male: 45 to 49 years',
    'Male: 50 to 54 years',
    'Male: 55 to 59 years',
    'Male: 60 to 64 years',
    'Male: 65 to 69 years',
    'Male: 70 to 74 years',
    'Male: 75 to 79 years',
    'Female: 10 to 14 years',
    'Male: 80 to 84 years',
    'Male: 85 to 89 years',
    'Male: 90 to 94 years',
    'Male: 95 to 99 years',
    'Male: 100 years and over',
    'Female: 0 to 04 years',
    'Female: 05 to 09 years',
    'Female: 15 to 19 years',
    'Female: 20 to 24 years',
    'Female: 25 to 29 years',
    'Female: 30 to 34 years',
    'Female: 35 to 39 years',
    'Female: 40 to 44 years',
    'Female: 45 to 49 years',
    'Female: 50 to 54 years',
    'Female: 55 to 59 years',
    'Female: 60 to 64 years',
    'Female: 65 to 69 years',
    'Female: 70 to 74 years',
    'Female: 75 to 79 years',
    'Female: 80 to 84 years',
    'Female: 85 to 89 years',
    'Female: 90 to 94 years',
    'Female: 95 to 99 years',
    'Female: 100 years and over'
]

characteristic_items = ['Single-detached house',
    'Apartment in a building that has five or more storeys',
    'Other attached dwelling',
    'Average household size',
    'Married or living common law',
    'Not married and not living common law',
    'Lone-parent census families in private households',
    'One-person households',
    'Two-or-more person non-census-family households',
    'Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)',
    'Non-immigrants',
    'Immigrants',
    'Owner',
    'Renter',
    'One person or fewer per room',
    'More than 1 person per room',
    'Suitable',
    'Not suitable',
    'No certificate, diploma or degree',
    'Secondary (high) school diploma or equivalency certificate',
    'Postsecondary certificate, diploma or degree',
    'Total population aged 15 years and over by work activity during the reference year - 25% sample data',
    'Did not work',
    'Worked',
    'Car, truck, van - as a driver',
    'Car, truck, van - as a passenger',
    'Public transit',
    'Walked',
    'Bicycle',
    'Other method',
    'With after-tax income',
    'Under $5,000',
    '$5,000 to $9,999',
    '$10,000 to $14,999',
    '$15,000 to $19,999',
    '$20,000 to $29,999',
    '$30,000 to $39,999',
    '$40,000 to $49,999',
    '$50,000 to $59,999',
    '$60,000 to $79,999',
    '$80,000 to $99,999',
    '$100,000 and over']

keep = ['Characteristic'] + characteristic_items

# this is to aggregate all ages
population = census2016.loc[:, census2016.columns.isin(pop_by_age_group)]
census2016 = census2016.loc[:, census2016.columns.isin(keep)]
census2016.head()

,Single-detached house,Apartment in a building that has five or more storeys,Other attached dwelling,Average household size,Married or living common law,Not married and not living common law,Lone-parent census families in private households,One-person households,Two-or-more person non-census-family households,"$100,000 and over",...,"Postsecondary certificate, diploma or degree",Total population aged 15 years and over by work activity during the reference year - 25% sample data,Did not work,Worked,"Car, truck, van - as a driver","Car, truck, van - as a passenger",Public transit,Walked,Bicycle,Other method
City of Toronto,"269,675","493,275","349,880",2.42,"1,181,360","1,152,075","152,595","359,960","68,010","116,680",...,1077935,"2,294,790","791,795","1,502,995","575,255","57,170","463,000","107,665","34,355","13,610"
Agincourt North,3345,2120,3645,3.16,14200,11075,1650,1350,290,245,...,7870,"25,000","11,060","13,940","7,155",930,"3,350",265,70,45
Agincourt South-Malvern West,2790,3145,2200,2.88,11205,9475,1360,1615,445,265,...,7890,"20,400","8,270","12,140","6,135",665,"2,985",280,35,65
Alderwood,2840,255,1525,2.6,5735,4550,670,1105,185,325,...,4350,"10,265","3,245","7,015","4,090",355,"1,285",195,65,65
Annex,645,8165,7125,1.8,12275,15890,895,7885,1430,"3,660",...,15390,"26,305","6,925","19,365","3,290",290,"6,200","3,200","1,675",225


In [131]:
# Rename the 0 to 04 year age groups for simplicity in code
population.rename(columns={'Male: 0 to 04 years': 'Male: 00 to 04 years', 'Female: 0 to 04 years': 'Female: 00 to 04 years'}, inplace=True)

In [132]:
for age_group in range(0, 101, 5):
    # Define column names for male and female
    male_col_name = f"Male: {age_group:02d} to {age_group + 4:02d} years"
    female_col_name = f"Female: {age_group:02d} to {age_group + 4:02d} years"
    
    if male_col_name in population.columns and female_col_name in population.columns:
        population[f"Population for {age_group:02d} to {age_group + 4:02d} years"] = population[male_col_name] + population[female_col_name]
        
        population.drop([male_col_name, female_col_name], axis=1, inplace=True)

In [133]:
population.head()

,Male: 100 years and over,Female: 100 years and over,Population for 00 to 04 years,Population for 05 to 09 years,Population for 10 to 14 years,Population for 15 to 19 years,Population for 20 to 24 years,Population for 25 to 29 years,Population for 30 to 34 years,Population for 35 to 39 years,...,Population for 50 to 54 years,Population for 55 to 59 years,Population for 60 to 64 years,Population for 65 to 69 years,Population for 70 to 74 years,Population for 75 to 79 years,Population for 80 to 84 years,Population for 85 to 89 years,Population for 90 to 94 years,Population for 95 to 99 years
City of Toronto,125,650,"69,89566,105","69,35065,680","64,94562,165","74,24071,285","97,41597,330","113,905119,040","108,895115,675","94,070102,240",...,"98,735103,670","88,14594,660","72,27081,600","60,36070,180","42,32051,285","32,73043,430","25,67034,965","15,66525,135","6,18513,500","1,2803,475"
Agincourt North,0,10,660620,695620,660595,840865,1015975,10151005,835935,680775,...,11601290,10601165,9251070,925985,590690,490575,380485,210350,100160,2560
Agincourt South-Malvern West,0,0,575530,540495,460475,780690,1000895,1045975,820835,625715,...,970975,850915,710795,630690,425450,350405,240350,155205,45100,2020
Alderwood,0,5,360320,270325,225255,285290,355310,355350,410430,455450,...,515515,540485,390400,300325,205210,155180,105210,65130,3570,105
Annex,5,25,445480,365400,325355,465550,12151520,20802265,16101675,10551040,...,920935,855915,755940,780950,640700,485565,335425,230345,80260,2590


In [134]:
# Merge together the two dataframes
census2016 = pd.concat([census2016, population], axis=1)

In [135]:
for col in census2016.columns:
    print(col)

Single-detached house
Apartment in a building that has five or more storeys
Other attached dwelling
Average household size
Married or living common law
Not married and not living common law
Lone-parent census families in private households
One-person households
Two-or-more person non-census-family households
$100,000 and over
$100,000 and over
$30,000 to $39,999
$40,000 to $49,999
$50,000 to $59,999
$20,000 to $29,999
With after-tax income
$20,000 to $29,999
$30,000 to $39,999
$40,000 to $49,999
$50,000 to $59,999
$5,000 to $9,999
$20,000 to $29,999
$30,000 to $39,999
$40,000 to $49,999
$50,000 to $59,999
$100,000 and over
$15,000 to $19,999
Under $5,000
$5,000 to $9,999
$10,000 to $14,999
$50,000 to $59,999
$100,000 and over
Under $5,000
$5,000 to $9,999
$10,000 to $14,999
$15,000 to $19,999
$50,000 to $59,999
$100,000 and over
Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)
Non-immigrants
Immigrants
Owner
Renter
One person or fewer per room
More than 

In [136]:
census2016.to_csv('data/census2016.csv')